# Joining NABD and NID

In [10]:
import geopandas as gp, pandas as pd, matplotlib as mpl
from pathlib import Path
gdrive = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo") #where shapefiles/csv live 
# gdrive2 = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Jen_Dams/Matching") #where shapefiles/csv live 

### Read in data

In [11]:
# NABD
nabd_dams = gp.read_file(gdrive/"nabd/nabd_fish_barriers_2012.shp")  #read in NABD from Drive
nabd_dams = pd.DataFrame(nabd_dams)
# 'OK02500' in nabd_dams.NIDID

# #### filtering
nabd_dams = nabd_dams.drop_duplicates(subset='NIDID', keep="first").reset_index(drop=True)  #drop everything after first duplicate
# nabd_dams = nabd_dams[['COMID', 'NIDID', 'Norm_stor', 'Max_stor','Year_compl', 'Purposes', 'geometry', 'Dam_name']]
# nabd_dams.Norm_stor = nabd_dams.Norm_stor * 1233.48

# # # NID
# # nid = pd.read_csv(gdrive/'other_dam_datasets/NID2019_U.csv', usecols=['NIDID', 'NORMAL_STORAGE', 'MAX_STORAGE', 'YEAR_COMPLETED', 'DAM_NAME', 'LATITUDE', 'LONGITUDE'])
nid = pd.read_csv(gdrive/'other_dam_datasets/NID2019_U.csv')

# #### filtering
# nid.NORMAL_STORAGE = nid.NORMAL_STORAGE * 1233.48   #convert units to cubic meters

In [22]:
'OK02500' in nabd_dams.NIDID.values
# test_str = nabd_dams.NIDID[1101]
# res = " " in test_str 
  
# # printing result  
# print("Does string contain spaces ? " + str(res)) 

True

### Figuring out what is missing

In [4]:
#Merge on NID
# nid_by_ID = nid.merge(nabd_dams, on = 'NIDID', how = 'left')
# nid_by_ID['COMID'] = nid_by_ID['COMID'].fillna(0)
# missing = nid_by_ID[nid_by_ID['COMID'] ==0]

# print('missing #',len(missing))

# large_missing = missing[missing['NORMAL_STORAGE']>=10**8]  #is it 10^8 or 10^6?
# print('large missing #', len(large_missing))
# large_missing.to_csv('large_dams_missing.csv')

### Make NID look like NABD, so we can add in missing dams

In [5]:
nid_gdf = gp.GeoDataFrame(nid, geometry=gp.points_from_xy(nid.LONGITUDE, nid.LATITUDE))
nid_gdf = nid_gdf[['NIDID', 'NORMAL_STORAGE', 'MAX_STORAGE', 'YEAR_COMPLETED', 'PURPOSES', 'geometry']]
nid_gdf = nid_gdf.rename(columns = {'NORMAL_STORAGE':'Norm_stor', 'MAX_STORAGE':'Max_stor','YEAR_COMPLETED':'Year_compl', 'PURPOSES':'Purposes'})
nid_gdf = nid_gdf.drop_duplicates(subset='NIDID', keep="first").reset_index(drop=True)
nid_gdf['COMID'] = 0
# nid_gdf.loc[21:24,'NIDID']
# len(nid_gdf)

### Get the dams we need to add to NABD in own df

In [23]:
# add_dams = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/archive/Rachel_testing/large_dams_to_add.csv', index_col = 0, usecols= [1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15])
add_dams = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/archive/Rachel_testing/large_dams_to_add.csv', index_col = 0, usecols= ['NIDID', 'DAM_NAME'])
add_dams = add_dams.drop_duplicates(subset='NIDID', keep="first").reset_index(drop=True)
# add_dams

dam_ls = []
for i in add_dams.NIDID:
    dam_ls.append(i)

nid_filtered = nid_gdf[nid_gdf['NIDID'] == dam_ls[0]] 

# for i in range(1,len(dam_ls)):
#     # print(i)
#     for j in range(len(nid_gdf.NIDID)):
#         if nid_gdf.loc[j,'NIDID'] == dam_ls[i]:
#             print(nid_gdf[nid_gdf['NIDID'] == dam_ls[i]])
            # nid_filtered = nid_filtered.append(nid_gdf[nid_gdf['NIDID'] == dam_ls[i]])

for j in range(len(nid_gdf.NIDID)):
    for i in range(1,len(dam_ls)):
        if nid_gdf.loc[j,'NIDID'] == dam_ls[i]:
            # print(nid_gdf[nid_gdf['NIDID'] == dam_ls[i]])
            nid_filtered = nid_filtered.append(nid_gdf[nid_gdf['NIDID'] == dam_ls[i]])

nid_filtered


KeyboardInterrupt: 

In [7]:
len(add_dams)

# nid_nhd_join = gp.sjoin(nid_gdf, flowlines, how="left", op='intersects')
# nid_nhd_join 
# nid_filtered

91

In [48]:
add_d

,NIDID
0,WY01290
1,ID00274
2,ID00279
3,SD01099
4,ID06126
...,...
86,NE02287
87,MA03263
88,OR00082
89,CA10303


In [8]:
nid_filtered.to_csv('dams_to_add.csv')

## Updating NIDID

In [24]:
nabd_updateID = nabd_dams.copy()
# nabd_updateID = nabd_updateID.drop_duplicates(subset='NIDID', keep="first").reset_index(drop=True)
# nabd_updateID.loc[10,'NIDID']
nabd_updateID

,COMMENT,NIDID,COMID,UNIQUE_STR,newX,newY,RecordID,Dam_name,Dam_former,STATEID,...,Spill_type,Num_locks,Len_locks,Wid_locks,Source,Condition,Cond_Date,Cond_desc,Spill_wid,geometry
0,1,AL00288,893441.0,1,-86.196373,31.424403,326.0,DONALDSON LAKE DAM,None,None,...,None,0.0,0.0,0.0,AL,None,None,None,0.0,POINT (-86.19637 31.42440)
1,1,AL01925,894119.0,2,-86.400374,31.170262,1679.0,CHARLES WOODHAM LAKE,None,None,...,None,0.0,0.0,0.0,AL,None,None,None,0.0,POINT (-86.40037 31.17026)
2,1,AL00648,895019.0,3,-86.299755,31.223052,641.0,JERRY ADAMS,None,None,...,None,0.0,0.0,0.0,AL,None,None,None,0.0,POINT (-86.29975 31.22305)
3,1,AL00652,895035.0,4,-86.450075,31.179562,642.0,CLIFTON MADDOX,None,AL36100004,...,U,0.0,0.0,0.0,AL,None,None,None,65.0,POINT (-86.45007 31.17956)
4,1,AL00641,895041.0,5,-86.346634,31.143277,638.0,JAMES CRAVEY,None,None,...,None,0.0,0.0,0.0,AL,None,None,None,0.0,POINT (-86.34663 31.14328)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51790,1,AK00189,41700279.0,52561,-149.551141,61.348327,0.0,None,None,None,...,None,0.0,0.0,0.0,None,None,None,None,0.0,POINT (-149.55114 61.34833)
51791,1,AK00029,41701413.0,52562,-149.924029,61.207976,0.0,None,None,None,...,None,0.0,0.0,0.0,None,None,None,None,0.0,POINT (-149.92403 61.20798)
51792,1,AK00060,62834739.0,52563,-149.457555,60.102455,0.0,None,None,None,...,None,0.0,0.0,0.0,None,None,None,None,0.0,POINT (-149.45755 60.10245)
51793,1,AK00019,62833387.0,52564,-148.077985,60.048443,0.0,None,None,None,...,None,0.0,0.0,0.0,None,None,None,None,0.0,POINT (-148.07799 60.04844)


In [25]:
# updated_ids = []
# nabd_ids = []
wrong_id = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/archive/Rachel_testing/large_dams_wrongID.csv', usecols= ['NIDID','DAM_NAME', 'NABD_NIDID'])
wrong_id
# wrong_id['NABD_NIDID'] = wrong_id['NABD_NIDID'].fillna(0)
wrong_id = wrong_id[wrong_id['NABD_NIDID'].notna()].reset_index(drop=True)

# # wrong_id.loc[:,'NIDID']

for j in range(len(nabd_updateID.NIDID)):
    for i in wrong_id.NABD_NIDID:
        if i in nabd_updateID.NIDID.values:
            for k in range(len(wrong_id.NIDID)):
                nabd_updateID.at[j,'NIDID'] = wrong_id.loc[k,'NIDID']


# for i in wrong_id.NABD_NIDID:
#     for j in range(len(nabd_updateID.NIDID)):
#         # print(i)
#         # print(nabd_updateID.loc[j,'NIDID'])
#         if i in nabd_updateID.NIDID.values:
#             # print(i)
#             for k in wrong_id.NABD_NIDID:
#                 nabd_updateID.loc[j,'NIDID'] = wrong_id.loc[k,'NIDID']
    # nabd_ids.append(i)
# for i in wrong_id.NIDID:
#     updated_ids.append(i)
# # if nabd_updateID.NIDID == 'OK02500':
# #     nabd_updateID.NIDID = 'OK82901'
# # nabd_updateID
# nabd_ids
# updated_ids



In [26]:
'AR00151' in nabd_updateID.NIDID.values
# nabd_updateID

False

In [17]:
# wrong_id.loc[0,'NIDID']
# nabd_updateID.loc['OK02500',:]
wrong_id.index

RangeIndex(start=0, stop=25, step=1)

In [30]:
# for i in nabd_updateID.NIDID:
#     if i in nabd_ids:
#         print(i)
#     #     nabd_updateID.loc[i,'NIDID'] = updated_ids[i]


## Figuring out where to add missing dams into workflow after processing in QGIS

In [19]:
add = gp.read_file(gdrive/'other_dam_datasets/dams_to_add_sjoin.csv')
nabd_dams = gp.read_file(gdrive/"nabd/nabd_fish_barriers_2012.shp")
# usecols= ['NIDID','Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'join_COMID']

In [43]:
add2 = add[['NIDID','Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'join_COMID', 'geometry']].rename(columns={'join_COMID':'COMID'})
nabd_dams = nabd_dams[['NIDID','Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'COMID', 'geometry']]
nabd_dams
# add2

,NIDID,Norm_stor,Max_stor,Year_compl,Purposes,COMID,geometry
0,AL00288,78.0,91.0,1952.0,R,893441.0,POINT (-86.19637 31.42440)
1,AL01925,75.0,85.0,1974.0,RI,894119.0,POINT (-86.40037 31.17026)
2,AL00648,75.0,81.0,1958.0,RI,895019.0,POINT (-86.29975 31.22305)
3,AL00652,36.0,70.0,1960.0,RF,895035.0,POINT (-86.45007 31.17956)
4,AL00641,106.0,123.0,1962.0,RI,895041.0,POINT (-86.34663 31.14328)
...,...,...,...,...,...,...,...
52451,AK00189,0.0,0.0,0.0,None,41700279.0,POINT (-149.55114 61.34833)
52452,AK00029,0.0,0.0,0.0,None,41701413.0,POINT (-149.92403 61.20798)
52453,AK00060,0.0,0.0,0.0,None,62834739.0,POINT (-149.45755 60.10245)
52454,AK00019,0.0,0.0,0.0,None,62833387.0,POINT (-148.07799 60.04844)


In [36]:
nabd_updated = add2.append(nabd_dams)

In [44]:
print('updated nabd', len(nabd_updated))
print('old nabd',len(nabd_dams))
print(len(nabd_updated)-len(nabd_dams))
len(add2)

updated nabd 52547
old nabd 52456
91


91

In [47]:
'NV10122' in nabd_dams.NIDID.values

False

In [54]:
add2['COMID'] = pd.to_numeric(add2['COMID'])
# df["a"] = pd.to_numeric(df["a"])
type(add2['COMID'][1])
type(nabd_dams['COMID'][1])

numpy.float64

## Junk

In [31]:
# wrong_id = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/archive/Rachel_testing/large_dams_wrongID.csv', index_col = 0, usecols= [1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15])
# print(wrong_id.Dam_name)

In [32]:
# nid = pd.read_csv(gdrive/'other_dam_datasets/NID2019_U.csv')
# nid.columns
# nid.NORMAL_STORAGE

In [33]:
# nid.MAX_STORAGE

In [45]:
# dams_adding = pd.read_csv(gdrive/'other_dam_datasets/dams_to_add_sjoin.csv', usecols=['join_COMID', 'NIDID', 'Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'WKT'])
# dams_adding = pd.read_csv(gdrive/'other_dam_datasets/dams_to_add_sjoin.csv')
dams_adding = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/dams_to_add_sjoin.csv', usecols=['join_COMID', 'NIDID', 'Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'WKT'])

#columns we need 
# ['COMID', 'NIDID', 'Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'geometry']

dams_adding = dams_adding.rename(columns = {'join_COMID':'COMID', 'WKT':'geometry'})
dams_adding


,geometry,NIDID,Norm_stor,Max_stor,Year_compl,Purposes,COMID
0,"MULTIPOLYGON (((-107.1866667 42.54793,-107.187...",WY01290,184405.0,184405.0,1937.0,HI,15951436
1,"MULTIPOLYGON (((-92.99916667 34.25111111,-92.9...",AR01205,654700.0,NaN,1969.0,CHORS,22001320
2,"MULTIPOLYGON (((-119.2113 37.2286,-119.2117894...",CA00434,89166.0,89800.0,1917.0,HR,948040323
3,"MULTIPOLYGON (((-121.8078 37.4952,-121.8082894...",CA01546,96850.0,NaN,2018.0,NaN,2806979
4,"MULTIPOLYGON (((-120.314 34.9862,-120.31448943...",CA10197,151050.0,398120.0,1958.0,CIO,17635380
...,...,...,...,...,...,...,...
86,"MULTIPOLYGON (((-104.0872 43.41674,-104.087689...",WY01299,701510.0,1473118.0,1951.0,CFHIRS,9553561
87,"MULTIPOLYGON (((-107.8825 43.2109,-107.8829894...",WY01378,152459.0,163976.0,1937.0,CFIR,12871409
88,"MULTIPOLYGON (((-109.05444 44.3825,-109.054929...",WY01380,334215.0,629025.0,1951.0,CI,4429427
89,"MULTIPOLYGON (((-108.9733333 43.8575,-108.9738...",WY01385,847000.0,872700.0,1911.0,CFIR,12842324


In [46]:
nabd = nabd_dams.copy()
nabd = nabd[['COMID', 'NIDID', 'Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'geometry']]

In [47]:
nabd = nabd.append(dams_adding)
nabd

,COMID,Max_stor,NIDID,Norm_stor,Purposes,Year_compl,geometry
0,893441.0,91.0,AL00288,78.0,R,1952.0,POINT (-86.19637299999994 31.42440300000004)
1,894119.0,85.0,AL01925,75.0,RI,1974.0,POINT (-86.40037399999994 31.17026200000004)
2,895019.0,81.0,AL00648,75.0,RI,1958.0,POINT (-86.29975499999995 31.22305200000006)
3,895035.0,70.0,AL00652,36.0,RF,1960.0,POINT (-86.45007499999991 31.17956200000009)
4,895041.0,123.0,AL00641,106.0,RI,1962.0,POINT (-86.34663399999995 31.14327700000007)
...,...,...,...,...,...,...,...
86,9553561.0,1473118.0,WY01299,701510.0,CFHIRS,1951.0,"MULTIPOLYGON (((-104.0872 43.41674,-104.087689..."
87,12871409.0,163976.0,WY01378,152459.0,CFIR,1937.0,"MULTIPOLYGON (((-107.8825 43.2109,-107.8829894..."
88,4429427.0,629025.0,WY01380,334215.0,CI,1951.0,"MULTIPOLYGON (((-109.05444 44.3825,-109.054929..."
89,12842324.0,872700.0,WY01385,847000.0,CFIR,1911.0,"MULTIPOLYGON (((-108.9733333 43.8575,-108.9738..."
